In [11]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import math

In [12]:
df = pd.read_csv('train_v2.csv')

In [13]:
df.shape

(1708337, 13)

In [14]:
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [15]:
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [16]:
for cols in json_columns:
    df_norm = pd.json_normalize([json.loads(val) for val in df[cols]]).fillna(0)
    df = pd.merge(df, df_norm, left_index=True, right_index=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 64 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   channelGrouping                      object
 1   customDimensions                     object
 2   date                                 int64 
 3   device                               object
 4   fullVisitorId                        object
 5   geoNetwork                           object
 6   hits_x                               object
 7   socialEngagementType                 object
 8   totals                               object
 9   trafficSource                        object
 10  visitId                              int64 
 11  visitNumber                          int64 
 12  visitStartTime                       int64 
 13  browser                              object
 14  browserVersion                       object
 15  browserSize                          object
 16  

In [18]:
df = df.drop(['socialEngagementType', 'browserVersion', 'browserSize', 'operatingSystemVersion',
              'mobileDeviceBranding', 'adwordsClickInfo.criteriaParameters', 'mobileDeviceModel', 
              'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 
              'language', 'screenColors', 'screenResolution', 'metro', 'city', 'adwordsClickInfo.isVideoAd',
              'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation',
              'campaign', 'keyword', 'referralPath', 'adwordsClickInfo.gclId',
              'device', 'geoNetwork', 'totals', 'trafficSource', 'customDimensions', 'hits_x', 'visits'
             ], axis=1)

In [19]:
df.replace(['(not set)', 'not available in demo dataset', '(none)'], np.nan, inplace=True)
df['isTrueDirect'].replace(0, 'False', inplace=True)
df['adContent'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.adNetworkType'].replace(0, np.nan, inplace=True)
df['adwordsClickInfo.slot'].replace(0, np.nan, inplace=True)

In [20]:
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['hits_y'] = df['hits_y'].astype(int)
df['sessionQualityDim'] = df['sessionQualityDim'].astype(int)
df['timeOnSite'] = df['timeOnSite'].astype(int)
df['transactions'] = df['transactions'].astype(int)
df['transactionRevenue'] = df['transactionRevenue'].astype(int)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)
df['visitId'] = df['visitId'].astype(str)

In [21]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['weekday'] = df['date'].dt.weekday
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 35 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   channelGrouping                 object        
 1   date                            datetime64[ns]
 2   fullVisitorId                   object        
 3   visitId                         object        
 4   visitNumber                     int64         
 5   visitStartTime                  int64         
 6   browser                         object        
 7   operatingSystem                 object        
 8   isMobile                        bool          
 9   deviceCategory                  object        
 10  continent                       object        
 11  subContinent                    object        
 12  country                         object        
 13  region                          object        
 14  hits_y                          int64         
 15

In [23]:
df['fullVisitorId'].duplicated().sum()

376561

In [34]:
df_2 = df.sample(frac = 0.05)

In [35]:
df_2.shape

(85417, 35)

In [36]:
df_2 = df_2.groupby(['fullVisitorId']).agg({'channelGrouping': lambda x:x.value_counts(dropna=False).index[0], 
                                         'date': 'max', 
                                         'visitId': lambda x:x.value_counts(dropna=False).index[0], 
                                         'visitNumber': 'max', 
                                         'visitStartTime': 'mean',
                                         'browser': lambda x:x.value_counts(dropna=False).index[0],
                                         'operatingSystem': lambda x:x.value_counts(dropna=False).index[0],
                                         'isMobile': lambda x:x.value_counts(dropna=False).index[0],
                                         'deviceCategory': lambda x:x.value_counts(dropna=False).index[0],
                                         'continent': lambda x:x.value_counts(dropna=False).index[0],
                                         'subContinent': lambda x:x.value_counts(dropna=False).index[0],
                                         'country': pd.Series.mode,
                                         'hits_y': 'sum',
                                         'pageviews': 'sum',
                                         'bounces': 'sum',
                                         'newVisits': 'sum',
                                         'sessionQualityDim': 'mean',
                                         'timeOnSite': 'sum',
                                         'transactions': 'sum',
                                         'transactionRevenue': 'sum',
                                         'totalTransactionRevenue': 'sum',
                                         'source': lambda x:x.value_counts(dropna=False).index[0],
                                         'weekday': 'max',
                                         'day': 'max',
                                         'month': 'max',
                                         'year': 'max'
                                        }).reset_index()

In [37]:
df_2.to_csv('df_clean.csv', index=False)